# Similitud Notebook
#### En este notebook podremos analizar con más facilidad la similitud entre las respuestas calificadas con mayor calificación con las de menor

In [1]:
# importo librerias
import spacy
import numpy as np
import pandas as pd
from hunspell import Hunspell

h = Hunspell('es_MX', hunspell_data_dir = 'C:/Python/Lib/site-packages/dictionaries')
spa_lex = spacy.load('es_core_news_md')

In [2]:
# con esto evito que se impriman 'errores'
import warnings
warnings.filterwarnings("ignore")

#### Importamos el CSV original como un dataframe

In [3]:
# df = pd.read_csv("C:/Users/Drablaguna/Desktop/UNAM/EvaluacionCognitiva/cuenta_de_palabras/WORDCOUNT_SECUNDARIA.csv")
df = pd.read_csv("C:/Users/Drablaguna/Desktop/UNAM/EvaluacionCognitiva/Bases de datos/Secundaria/SECUNDARIA_TODO.csv")
#txt_file = open("C:/Users/Drablaguna/Desktop/ResFile.txt", "a+")
#txt_file.close()

#### Funciones

In [4]:
# funciones para eliminar caracteres de puntuacion
def evalChar(x):
    if x in [",",".",":",";","`","'",'"',"(",")","-","_","~","/","?","¿","=","[","]","\n","\r"]:
        x = ""
    return x
def cleanString(x):
    s = map(evalChar, x)
    s = "".join(list(s))
    return s

In [5]:
# funcion para inicializar el diccionario de resultados, para tener resultados diferentes por prueba
def start_dict():
    dic = {
    'token_base': [],      # tokens originales
    'token_toCompare': [], # token con el que se comparo
    'similarity': [],      # valor de la similitud de ambos
    'total_sim_mean': 0    # promedio similitud por frase
    }
    return dic

In [6]:
# funcion para analizar
def analize(base, comparer):
    dic = start_dict()
    for token_base in base:
        for token_toCompare in comparer:
            simil = token_base.similarity(token_toCompare)
            dic['token_base'].append(token_base.text)
            dic['token_toCompare'].append(token_toCompare.text)
            dic['similarity'].append(simil)
    # genero el promedio de similitud que fue extraido de la sim de cada token con token
    dic['total_sim_mean'] = np.mean(dic['similarity'])
    return dic

In [7]:
# funcion para analizar rapido y mostrar similitud
def quickize(master_row, slave_row):
#     master_row = checkSentence(master_row,h)
#     slave_row = checkSentence(slave_row,h)
    master_row = spa_lex(master_row)
    slave_row = spa_lex(slave_row)
    dic = analize(master_row, slave_row)
    return dic

# funcion para imprimir las palabras que se compararon y sus similitudes
def print_zip(dic):
    print(list(zip(dic['token_base'], dic['token_toCompare'], dic['similarity'])))

In [8]:
# funcion para elegir la palabra correcta entre sus sugerencias
def checkWord(word, hun_dic, original_sentence):
    if not hun_dic.spell(word): # si la palabra NO se encuentra en el diccionario hunspell
        n = 0
        a = hun_dic.suggest(word)
        print("=> " + original_sentence)
        for x in a:
            print(str(n) + ": " + x)
            n+=1
        # se puede ingresar la respuesta correcta con el id o escribiendola manualmente
        correct_id_word = input("("+word+") ID respuesta correcta o ingresa la palabra: ")
        if correct_id_word.isdigit():
            correct = a[int(correct_id_word)]
        else:
            correct = correct_id_word
    else: # si la palabra se encuentra en el diccionario hunspell
        correct = word
    return correct.strip().lower()

# funcion para checar una oracion usando la funcion de escoger la palabra correcta
def checkSentence(s_string, hun_dic):
    new_sentence = []
    for token in s_string.split(" "): # se evaluara cada token si es una palabra existente o no
        correct_word = checkWord(token, hun_dic, s_string)
        if correct_word != "": # se agregara a la oracion nueva solo si es algo
            new_sentence.append(correct_word)
    #print("\n")
    return " ".join(new_sentence)
        
# s = "porque sii mas que nada"
# s = checkSentence(s,h)
# print(s)

# Resultado (SECU_TODO_CLEAN)

In [ ]:
# promedio similitud entre respuestas
#print('\n')
#print(list(zip(results['token_base'], results['token_toCompare'], results['similarity'])))

# Resultado (WORDCOUNT_SECUNDARIA)

In [ ]:
# aqui se han retirado las STOPWORDS
# de acuerdo al siguiente criterio de quickPlotter.py usando Spacy
# if token.is_stop or token.is_punct or token.is_quote or len(token) == 1
#print('\n')
#print(list(zip(results['token_base'], results['token_toCompare'], results['similarity'])))

### Dataframe base

In [ ]:
# WORDCOUNT SECUNDARIA ===========================================================
# ans_col = "explica_lo_que_quiere_decir_mi_hermanito_es_una_pelota_de_gritos_clean_ans"
# eval_col = "explica_lo_que_quiere_decir_mi_hermanito_es_una_pelota_de_gritos_eval"

# SECU TODO CLEAN ================================================================
ans_col = "explica_lo_que_quiere_decir_mi_hermanito_es_una_pelota_de_gritos"
eval_col = "a5"

# obtengo los valores de respuestas limpias y sus evaluaciones en un dataframe
df_base = df[[ans_col, eval_col]]

### Dataframes evaluados a  3, 2, 1 y 0

In [ ]:
# df head testing

#df_three.head(60)
#df_three.shape

#df_two.head(3)
#df_two.shape

#df_one.head(3)
#df_one.shape

#df_zero.head(3)
#df_zero.shape

print('3 => '+str(len(df_three)))
print('2 => '+str(len(df_two)))
print('1 => '+str(len(df_one)))
print('0 => '+str(len(df_zero)))

In [20]:
# funcion para imprimir rapido resultados de similitudes en una columna
"""
def print_df_res(df_master, df_slave, ans_col):
    counter = 1
    total_mean = []
    for row in df_master[ans_col]:
        master_row = cleanString(row) # la row limpia sin [] ni comas
        print("============================== "+master_row.upper()+" ==============================")
        for line in df_slave[ans_col]:
            slave_row = cleanString(line)
            if slave_row != "": # para evitar evaluaciones donde la fila este vacia
                r = quickize(master_row, slave_row)
                total_mean.append(r['total_sim_mean'])
                print(str(counter)+': '+str(r['total_sim_mean']))
                counter+=1
    print(np.mean(total_mean))
""" 

# funcion para evaluar 1 respuesta de 3 con todo
def print_df_res(master_row, df_slave, ans_col):
    counter = 1
    total_mean = []
    master_row = cleanString(master_row) # la row limpia sin [] ni comas
    print("============================== "+master_row.upper()+" ==============================")
    for line in df_slave[ans_col]:
        slave_row = cleanString(line)
        if slave_row != "": # para evitar evaluaciones donde la fila este vacia
            r = quickize(master_row, slave_row)
            total_mean.append(r['total_sim_mean'])
            print(str(counter)+': '+str(r['total_sim_mean']))
            counter+=1
    print(f"\n{np.mean(total_mean)}")

def quickPrint(df, ans_col, eval_col, n1, n2):
    df_base  = df[[ans_col, eval_col]]
    
    df_three = df_base.loc[df_base[eval_col] == 3]
    df_two   = df_base.loc[df_base[eval_col] == 2]
    df_one   = df_base.loc[df_base[eval_col] == 1]
    df_zero  = df_base.loc[df_base[eval_col] == 0]
    
    three_ans= "por que hace mucho ruido"
    
    # 3 con 3
    if n1 == 3 and n2 == 3:
        print_df_res(three_ans, df_three, ans_col)
        
    # 3 con 2
    if n1 == 3 and n2 == 2:
        print_df_res(three_ans, df_two, ans_col)
        
    # 3 con 1
    if n1 == 3 and n2 == 1:
        print_df_res(three_ans, df_one, ans_col)
        
    # 3 con 0
    if n1 == 3 and n2 == 0:
        print_df_res(three_ans, df_zero, ans_col)
    
    
    # 2 con 2
    if n1 == 2 and n2 == 2:
        print_df_res(df_two, df_two, ans_col)
        
    # 2 con 1
    if n1 == 2 and n2 == 1:
        print_df_res(df_two, df_one, ans_col)
        
    # 2 con 0
    if n1 == 2 and n2 == 0:
        print_df_res(df_two, df_zero, ans_col)
        
    
    # 1 con 1
    if n1 == 1 and n2 == 1:
        print_df_res(df_one, df_one, ans_col)
        
    # 1 con 0
    if n1 == 1 and n2 == 0:
        print_df_res(df_one, df_zero, ans_col)
        
    
    # 0 con 0
    if n1 == 0 and n2 == 0:
        print_df_res(df_zero, df_zero, ans_col)

In [21]:
quickPrint(df, "explica_lo_que_quiere_decir_mi_hermanito_es_una_pelota_de_gritos", "a5", 3,3)

============================== HACE MUCHO RUIDO ==============================
1: 0.7908097902933756
2: 0.692073668870661
3: 0.63441604
4: 0.6455944726864496
5: 0.34760058
6: 0.74377304
7: 0.5172215
8: 0.69604695
9: 0.6575033
10: 0.5862435079697106
11: 0.6521628
12: 0.5869673
13: 0.66962874
14: 0.584592
15: 0.5774565090735754
16: 0.6254338712403269
17: 0.671272
18: 0.7222386598587036
19: 0.47466193238894144
20: 0.48390064
21: 0.7186003973086675
22: 0.7222386598587036
23: 0.690108597278595
24: 0.6717217
25: 0.7109221
26: 0.7186003973086675
27: 0.6549445961912473
28: 0.653054271141688
29: 0.60645413
30: 0.72907513
31: 0.6909651
32: 0.4823807458082835
33: 0.5987687806288401
34: 0.51906234
35: 0.5546177377303442
36: 0.66337305
37: 0.6693491
38: 0.6962573985258739
39: 0.7160229223469893
40: 0.59738666
41: 0.67908418054382
42: 0.124557555
43: 0.6672377660870552
44: 0.6849458
45: 0.69102937
46: 0.687318719095654
47: 0.667174713479148
48: 0.65641123
49: 0.6735578995375406
50: 0.68622637
51: 0.

In [22]:
quickPrint(df, "explica_lo_que_quiere_decir_mi_hermanito_es_una_pelota_de_gritos", "a5", 3,2)

============================== HACE MUCHO RUIDO ==============================
1: 0.683357797563076
2: 0.6514897
3: 0.5970531577865282
4: 0.7222386598587036
5: 0.6460457
6: 0.71435386
7: 0.57668422801154
8: 0.65846986
9: 0.5761179576317469
10: 0.6768642
11: 0.64144737
12: 0.7351025
13: 0.6202015692279452
14: 0.64039916
15: 0.5892019
16: 0.5865907
17: 0.66847384
18: 0.7957089874479506
19: 0.6776431
20: 0.7055505
21: 0.6751172
22: 0.72379535
23: 0.7287636756896972
24: 0.7156275
25: 0.73003954
26: 0.5840103112989001
27: 0.7101863
28: 0.30391842375199
29: 0.5834636842211087
30: 0.6865756655732791
31: 0.71977836

0.655621636235067


In [24]:
quickPrint(df, "explica_lo_que_quiere_decir_mi_hermanito_es_una_pelota_de_gritos", "a5", 3,1)

============================== HACE MUCHO RUIDO ==============================
1: 0.049953356
2: 0.58595693
3: 0.7094902495543162
4: 0.33200097
5: 0.6765080044666926
6: 0.7896245
7: 0.6635911998649439
8: 0.6545005520184835
9: 0.7896245
10: 0.6279823
11: 0.6869056
12: 0.49646565
13: 0.3936297992865245
14: 0.0
15: 0.12753445
16: 0.27429616
17: 0.79458857
18: 0.12753445
19: 0.6969177
20: 0.6821304
21: 0.61829966
22: 0.590347
23: 0.5566716069976488
24: 0.55452156
25: 0.7222386598587036
26: 0.70108384
27: 0.7222386598587036
28: 0.51964974
29: 0.6375952
30: 0.6628801
31: 0.5271738059278848
32: 0.6510601927836736
33: 0.69385856
34: 0.6668325
35: 0.50403893
36: 0.53225344
37: 0.6407855543352309
38: 0.14086904
39: 0.5503851366746757

0.5474876567976648


In [23]:
quickPrint(df, "explica_lo_que_quiere_decir_mi_hermanito_es_una_pelota_de_gritos", "a5", 3,0)

============================== HACE MUCHO RUIDO ==============================
1: 0.0
2: 0.58271307
3: 0.4782461
4: 0.0
5: 0.0

0.2121918320655823
